In [ ]:
'''
-- complete set data extraction query
select i.id, i.initial_value--, q.select_no
from polls_image i, polls_question q
where i.question_id = q.id
and q.id in (
	select id 
	from polls_question 
	where survey_id in (
		select qst.survey_id 
		from polls_image img, polls_question qst
		where img.question_id = qst.id
		group by survey_id, qst.survey_id
		having count(img.id)=50
	)
)
and train_yn=0
order by id asc
'''

In [54]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def computeDiffPair(x_val, y_val, isTrain):
    i = 0
    k = 0
    x_diff=list()
    y_diff=list()
    while( i <len(y_val)-1):
        if(y_val[i]==2):
            y_pref=-1
        else:
            y_pref = 1
        
        
        x_diff.append(x_val[i]-x_val[i+1])
        y_diff.append(y_pref)
        if(isTrain):
            if(y_diff[-1] != (-1)**k):
                y_diff[-1] *= -1
                x_diff[-1] *= -1
       
        i +=2
        k+=1
    return x_diff, y_diff
                      
x_train = pd.read_csv('C:/Users/aeroplan/Desktop/res_train_x.txt', header=None, sep='\t').astype('float64')
x_train = x_train.loc[:, 1:].as_matrix()

y_train = pd.read_csv('C:/Users/aeroplan/Desktop/res_train_y.txt', header=None).astype('int32')
y_train = y_train.loc[:, 1:].as_matrix()

x_train, y_train = computeDiffPair(x_train, y_train, True)

                      
clf = SVC(kernel='rbf', gamma=0.016 ) 
clf_fit = clf.fit(x_train, y_train)
    
x_test = pd.read_csv('C:/Users/aeroplan/Desktop/res_test_x.txt', header=None, sep='\t').astype('float64')
x_test = x_test.loc[:, 1:].as_matrix()
y_test = pd.read_csv('C:/Users/aeroplan/Desktop/res_test_y.txt', header=None).astype('int32')
y_test = y_test.loc[:, 1:].as_matrix()

x_test,y_test = computeDiffPair(x_test, y_test, False)
y_predict = clf_fit.predict(x_test)
print("predicted: "+str(y_predict))
print("test: "+str(y_test))
accuracy_score(y_test, y_predict)

predicted: [-1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1]
test: [1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1]


0.5